In [25]:
from ts_transform import dataset_to_sktime
import pickle
import numpy as np
from sktime.transformers.series_as_features.rocket import Rocket
from utils import progressBar

In [23]:
def apply_rocket_kernels(sktime_dataset, rocket):

    data_features = []

    for i, sample in enumerate(sktime_dataset):
        
        progressBar(i, len(sktime_dataset), 40)
        data_features.append(rocket.transform(sample))

    data_features = np.concatenate(data_features, axis=0)
    
    return data_features

In [5]:
dataset_names = [("POLLUTION", 5, 25)]
dataset_name, window_size, task_size = dataset_names[0]

train_data = pickle.load(  open( "../Data/TRAIN-"+dataset_name+"-W"+str(window_size)+"-T"+str(task_size)+"-NOML.pickle", "rb" ) )
validation_data = pickle.load( open( "../Data/VAL-"+dataset_name+"-W"+str(window_size)+"-T"+str(task_size)+"-NOML.pickle", "rb" ) )
test_data = pickle.load( open( "../Data/TEST-"+dataset_name+"-W"+str(window_size)+"-T"+str(task_size)+"-NOML.pickle", "rb" ) )
test_data_ML = pickle.load( open( "../Data/TEST-"+dataset_name+"-W"+str(window_size)+"-T"+str(task_size)+"-ML.pickle", "rb" ) )

In [6]:
train_data_sktime = dataset_to_sktime(train_data)

Transforming dataset to sktime format...


In [ ]:
num_kernels = 1000
rocket = Rocket(num_kernels=num_kernels)
rocket.fit(train_data_sktime[0])
train_features = apply_rocket_kernels(train_data_sktime, rocket)


In [29]:
pickle_out = open("../Data/TRAIN-POLLUTION-W5-T25-NOML-ROCKET-1000.pickle", "wb")
pickle.dump(train_features, pickle_out)
pickle_out.close()

In [9]:
validation_data_sktime = dataset_to_sktime(validation_data)


Transforming dataset to sktime format...


NameError: name 'val_data_sktime' is not defined

In [26]:
val_features = apply_rocket_kernels(validation_data_sktime, rocket)


In [30]:
pickle_out = open("../Data/VAL-POLLUTION-W5-T25-NOML-ROCKET-1000.pickle", "wb")
pickle.dump(val_features, pickle_out)
pickle_out.close()

In [31]:
model = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1]).fit(train_features, train_data.y)


TypeError: predict() takes 2 positional arguments but 3 were given

In [32]:
y_pred = model.predict(val_features)
score = mae(y_pred, validation_data.y)
print(score)

0.04104682008330958
